In [5]:

import pandas as pd
import numpy as np
import sklearn as sk

In [15]:
pwd

'/Users/mihaelanemes/Desktop/s2ds/data/Data'

In [14]:
cd /Users/mihaelanemes/Desktop/s2ds/data/Data/

/Users/mihaelanemes/Desktop/s2ds/data/Data


In [16]:
food_and_you_wales = pd.read_csv("food_and_you_all_waves_wales.csv")
print (food_and_you_wales)

      SerialNo  RespSex  age_dv  bhhsize2  below6  below16  marstat2  hhdinc  \
0       100027        2      -1        -1      -1        1         1       3   
1       100037        2      -1        -1      -1        2         1      -9   
2       100129        2      -1        -1      -1        2         1       4   
3       100130        1      -1        -1      -1        2         1       4   
4       100285        2      -1        -1      -1        2         1       4   
5       100314        1      -1        -1      -1        1         1       2   
6       100406        2      -1        -1      -1        2         2       2   
7       100591        2      -1        -1      -1        2         1      -9   
8       100815        2      -1        -1      -1        2         2       1   
9       100969        1      -1        -1      -1        2         2      -9   
10      101144        2      -1        -1      -1        1         1      -9   
11      101169        2      -1        -

In [18]:
# save outcome variable in new object

food_poisoning_last_year = pd.DataFrame(food_and_you_wales, columns=["bpoison"])


In [19]:
# return numpy array from df

food_and_you_scaled = food_and_you_wales.values
food_and_you_scaled


array([[ 1.00027e+05,  2.00000e+00, -1.00000e+00, ..., -1.00000e+00,
        -1.00000e+00, -1.00000e+00],
       [ 1.00037e+05,  2.00000e+00, -1.00000e+00, ..., -1.00000e+00,
        -1.00000e+00, -1.00000e+00],
       [ 1.00129e+05,  2.00000e+00, -1.00000e+00, ..., -1.00000e+00,
        -1.00000e+00, -1.00000e+00],
       ...,
       [ 5.14342e+05,  1.00000e+00,  3.00000e+00, ..., -1.00000e+00,
        -1.00000e+00, -1.00000e+00],
       [ 5.14440e+05,  2.00000e+00,  6.00000e+00, ..., -1.00000e+00,
        -1.00000e+00, -1.00000e+00],
       [ 5.14509e+05,  2.00000e+00,  2.00000e+00, ...,  0.00000e+00,
         0.00000e+00,  1.00000e+00]])

In [20]:
# calculate length of rows 

no_rows = len (food_and_you_scaled)
no_rows

1757

In [21]:
# calculate length of columns 

no_columns = len (food_and_you_scaled [0])
no_columns

502

In [22]:
# remove values that are 'refused to answer', 'unknown' etc
# remove by column?

for column in food_and_you_scaled 
    if food_and_you_scaled [column] = -1 or food_and_you_scaled [column] = -8 or 
    food_and_you_scaled [column] = -9
        df.replace ('-1', np.nan)
        

    

SyntaxError: invalid syntax (<ipython-input-22-ce1dc14a9ec5>, line 4)

In [34]:
# normalise each column between 0 and 1
from sklearn import preprocessing



[[0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.41265001e-05 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.46090301e-04 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.99597087e-01 0.00000000e+00 5.00000000e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99833527e-01 1.00000000e+00 8.75000000e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 1.00000000e+00 3.75000000e-01 ... 5.00000000e-01
  5.00000000e-01 1.00000000e+00]]


In [ ]:
# create new function to normalise by column between 0 and 1
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


In [29]:
# import regression model
## could use Recursive feature elimination (?)
from sklearn import linear_model
from sklearn import LinearRegression

ImportError: cannot import name 'LinearRegression' from 'sklearn' (/Users/mihaelanemes/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py)

In [27]:
# run linear regression model
food_poisoning_model = LinearRegression().fit(norm_food_and_you, food_poisoning_last_year)

NameError: name 'LinearRegression' is not defined